# Setup

## Instructions

This notebook utilizes the Kaggle API. Follow these steps in order to get the necessary credentials to continue:

1. Sign up for a Kaggle account at https://www.kaggle.com.
2. Go to the 'Account' tab of your user profile - ```https://www.kaggle.com/{username}/account```.
3. Select 'Create New API Token' under 'API' section.
    - This will trigger the download of kaggle.json, a file containing your API credentials. 
4. Place this file in the location:
    - ~/.kaggle/kaggle.json (for macOS/unix)
    - C:\Users\<Windows-username>\.kaggle\kaggle.json (for Windows) 
    - You can check the exact location, sans drive, with echo %HOMEPATH%). 
    - You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to:
        - $KAGGLE_CONFIG_DIR/kaggle.json (for macOS/unix)
        - %KAGGLE_CONFIG_DIR%\kaggle.json (for Windows)

## Additional Information

Documentation Guide:
- Kaggle API ([Kaggle](https://www.kaggle.com/docs/api))
- Kaggle API ([GitHub](https://github.com/Kaggle/kaggle-api)) 

## Imports

In [1]:
# Import kaggle, installing if necessary
try:
    import kaggle
except ImportError as e:
    !pip3 install kaggle
    import kaggle
    
import subprocess # Used to run unix commands
import pandas as pd # For storing/manipulating command data
from io import StringIO # Lets us read csv string output from command into DataFrame
import json # Reading back the metadata files
from tqdm import tqdm # Gives status bar on loop completion
import itertools # For efficient looping over queries
import os # Exporting saved results
from collections import OrderedDict

# Data wrangling

## Getting/extracting dataset names

In [2]:
def get_search_outputs(search_terms, search_types, save=False):
    """
    Call the Kaggle API for each search term and search type. 
    Results are retured in results['{term}_{type}'] = df
    
    Params:
    - search_terms (list-like): collection of search terms to query over
    - search_types (list-like): collection of search types to query over
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain len(search_terms) * len(search_types) strings
            For ordering, the queries are done using itertools.product(search_terms, search_types)
    
    Returns:
    - results (dict): dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    num_searches = len(search_terms) * len(search_types)
    results = OrderedDict()
    
    for search_term, search_type in itertools.product(search_terms, search_types):
        results[(search_term, search_type)] = get_search_output(search_term, search_type)
        
    return results

In [20]:
def get_search_output(search_term, search_type):
    """
    Calls the Kaggle API with the specified search term and returns the search output results.
    
    Params:
    - search_term (str): keyword to seach for
    - search_type (str): objects to search over (must be either datasets or kernels)
    
    Returns:
    - df (pandas.DataFrame): DataFrame containing the output of the search query
    """
    
    # Make sure our input is valid
    assert isinstance(search_type, str), 'Search term must be a string'
    assert search_type in ('datasets', 'kernels'), 'Search can only be conducted over datasets or kernels'
    
    # Set search variables
    page_idx = 1
    search_output = ''
    cumulative_output = ''
    completion_phrase = f'No {search_type} found\n'
    
    # Pulls the records for a single page of datasets for the given search term
    # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
    search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                     '-s', f'"{search_term}"', 
                                     '-p', str(page_idx)], 
                                    capture_output=True).stdout.decode()

    # Once we no longer see new output, we stop
    while search_output != completion_phrase:
        # Accumulate the output
        cumulative_output = cumulative_output + search_output

        # Increments the page count for searching
        page_idx += 1

        # Pulls the records for a single page of datasets for the given search term
        # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
        search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                         '-s', f'"{search_term}"', 
                                         '-p', str(page_idx)], 
                                        capture_output=True).stdout.decode()

        # Remove header row
        if search_output != completion_phrase:
            search_output = '\r\n'.join(search_output.split('\r\n')[1::])
    
    # Convert search output to DataFrame
    search_df = _convert_string_csv_output_to_dataframe(cumulative_output)
    
    # Rename columns to match names present in metadata df
    search_df.rename(columns={'ref': 'id', 
                              'downloadCount': 'totalDownloads', 
                              'voteCount': 'totalVotes'}, 
                     inplace=True)
        
    return search_df

In [4]:
def _convert_string_csv_output_to_dataframe(output):
    """
    Given a string variable in csv format, returns a Pandas DataFrame
    
    Params:
    - output (str): csv-styled string to be converted
    
    Returns:
    - df (pandas.DataFrame): DataFrame consisting of data from 'output' string variable
    """
    
    # Create DataFrame of results
    output = StringIO(output)
    df = pd.read_csv(output)
    
    return df

### Perform Search

In [5]:
search_terms = ['korea']
search_types = ['datasets', 'kernels']

In [21]:
search_output_dict = get_search_outputs(search_terms, search_types)

In [8]:
sample_key = (search_terms[0], search_types[0])
sample_df = search_output_dict[sample_key]

In [9]:
sample_df.head()

,ref,title,size,lastUpdated,downloadCount,voteCount,usabilityRating
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,83497,1466,1.000000
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9874,311,1.000000
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1035,24,1.000000
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,582,15,0.970588
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5533,107,0.750000


## Pulling dataset metadata

Note: Unable to find a way to store metadata in memory as opposed to saving file, but this workaround appears to be functional.

In [10]:
def _retrieve_object_json(object_path):
    """
    Queries Kaggle for metadata json file & returns the json data as a dictionary
    
    Params:
    - object_path (str): path for the dataset
    
    Returns:
    - metadata_dict (dict): dictionary containing json metadata
    """
    
    # Download the metadata
    subprocess.run(['kaggle', 'datasets', 'metadata', object_path])

    # Access the metadata and load it in as a dictionary
    with open('dataset-metadata.json') as file:
        json_data = json.load(file)
        
    return json_data

In [11]:
def get_metadata(object_paths):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths (str/list-like): string or list of strings containing the paths for the objects
    
    Returns:
    - metadata_df (pandas.DataFrame): DataFrame containing metadata for the requested datasets
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
        
    # Run first query
    json_data = _retrieve_object_json(object_paths[0])
        
    # Create DataFrame to store metadata in, using columns found in first query, and then add query info
    metadata_df = pd.DataFrame(columns=json_data.keys(), dtype=object)
    metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    # Pulls metadata information for each dataset found above
    for object_path in tqdm(object_paths[1::]):
        # Download & load the metadata
        json_data = _retrieve_object_json(object_path)

        # Store the metadata into our DataFrame created above
        metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    return metadata_df

### Perform Metadata Extraction

In [23]:
## Extract IDs from DataFrame, and returns as list of strings
metadata_dict = OrderedDict()

for query, df in search_output_dict.items():
    print(f'Retrieving {query} metadata')
    # Create object paths
    _, search_type = query
    object_paths = df.id.values
    
    metadata_dict[query] = get_metadata(object_paths)

Retrieving ('korea', 'datasets') metadata


100%|██████████| 176/176 [02:47<00:00,  1.05it/s]


Retrieving ('korea', 'kernels') metadata


100%|██████████| 1019/1019 [33:23<00:00,  1.97s/it]   


## Combining Results

In [13]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on, save=False):
    """
    Merges together all of the search and metadata DataFrames by the given 'on' key
    
    Params:
    - search_dict (dict): dictionary of search output results
    - metadata_dict (dict): dictionary of metadata results
    - on (str/list-like): column name(s) to merge the two dicts on
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain enough strings (one per query; len(search_terms) * len(search_types))
            
    Returns:
    - df_dict (OrderedDict): OrderedDict containing all of the merged search/metadata dicts
    """

    # Make sure the dictionaries contain the same searches
    assert search_dict.keys() == metadata_dict.keys(), 'Dictionaries must contain the same searches'
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')

    # Ensure the on variable is proper
    try:
        assert len(on) == 2 or isinstance(on, str)
        if (len(on) == 2) and (not isinstance(on, str)):
            left_on, right_on = on
            on = None
    except:
        raise ValueError('Incorrect value of "on" passed')
        
    # Merge the DataFrames
    df_dict = OrderedDict()
    for (query_key, search_df), (query_key, metadata_df), save_loc in zip(search_dict.items(), 
                                                                          metadata_dict.items(), 
                                                                          save):

        #Merge small version of "full" dataframe with "detailed" dataframe
        if on: # only one value to merge on
            df_all = pd.merge(search_df, metadata_df, on=on, how='inner')
        else:
            df_all = pd.merge(search_df, metadata_df, left_on=left_on, right_on=right_on)
            
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'figshare')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure kaggle directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError('Save type must be bool or str')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

In [ ]:
left_on = ['ref', 'title', 'usabilityRating', 'downloadCount', 'title']
right_on = ['id', 'title', 'usabilityRating', 'totalDownloads', 'title']
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict, on='id')

In [14]:
output_df = search_output_dict[('korea', 'datasets')]
metadata_df = metadata_dict[('korea', 'datasets')]

In [17]:
output_df.head()

,ref,title,size,lastUpdated,downloadCount,voteCount,usabilityRating
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,83497,1466,1.000000
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9874,311,1.000000
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1035,24,1.000000
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,582,15,0.970588
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5533,107,0.750000


In [19]:
metadata_df.head()

,id,id_no,datasetId,datasetSlug,ownerUser,usabilityRating,totalViews,totalVotes,totalDownloads,title,subtitle,description,isPrivate,keywords,licenses,collaborators,data
0,kimjihoo/coronavirusdataset,527325,527325,coronavirusdataset,kimjihoo,1.000000,490936,1466,83497,[NeurIPS 2020] Data Science for COVID-19 (DS4C),DS4C: Data Science for COVID-19 in South Korea,### A portion of our dataset has been accepted...,False,"[universities and colleges, biology, data visu...",[{'name': 'CC-BY-NC-SA-4.0'}],"[{'username': 'kjm0623v', 'role': 'writer'}, {...",[]
1,bappekim/air-pollution-in-seoul,576393,576393,air-pollution-in-seoul,bappekim,1.000000,81243,311,9874,Air Pollution in Seoul,Air Pollution Measurement Information in Seoul...,### Context\nThis dataset deals with air pollu...,False,"[earth and nature, environment, pollution]",[{'name': 'CC-BY-SA-4.0'}],[],[]
2,bappekim/south-korea-visitors,692628,692628,south-korea-visitors,bappekim,1.000000,5843,24,1035,South Korea Visitors,Foreign visitors into South Korea,### Context\n\nThis dataset deals with the vis...,False,"[global, travel]",[{'name': 'CC-BY-SA-4.0'}],[],[]
3,hongsean/korea-income-and-welfare,1046735,1046735,korea-income-and-welfare,hongsean,0.970588,3018,15,582,Korea Income and Welfare,Where Korea wealth come from?,![](https://www.googleapis.com/download/storag...,False,"[income, education, social issues and advocacy...",[{'name': 'CC0-1.0'}],[],[]
4,bryanpark/korean-single-speaker-speech-dataset,19829,19829,korean-single-speaker-speech-dataset,bryanpark,0.750000,83396,107,5533,Korean Single Speaker Speech Dataset,KSS Dataset: Korean Single Speaker Speech Dataset,"# [Updated on September 28, 2019] KSS Dataset:...",False,"[languages, arts and entertainment, education]",[{'name': 'CC-BY-NC-SA-4.0'}],[],[]
